# Infinitesimal Rigidity

In this notebook, we are going to be exploring the conept of *infinitesimal rigidity*. In particular, We will be looking at

* Trivial and non-trivial flexes
* The Rigidity Matrix

## Imports and helpful functions

Before we begin, we first import packages and define some functions that we will use throughout.


In [1]:
from sympy import Matrix, solve, symbols
import numpy as np
from scipy import linalg

def Vector(*args):
    return Matrix([[x] for x in args])

## Trivial and non-trivial flexes

$\newcommand{\iprod}[2]{\left\langle #1,#2\right\rangle}$

Recall that, for a framework $(G,p)$ in dimension $d$, a vector configuration $v$ of $n$ vectors in $\mathbb{R}^d$ is an **infinitesimal flex** of $(G,p)$ if
$$
    \iprod{p_j - p_i}{v_j - v_i} = 0
    \qquad \text{(all edges $ij\in E$)}.
$$
An infinitesimal flex is **trivial** if the stronger condition
$$
    \iprod{p_j - p_i}{v_j - v_i} = 0
    \qquad \text{(all vertices $i\neq j\in V$)}
$$
holds.

As the name might sugest, all frameworks permit trivial motions. Therefore, we are usually interested in the non-trivial case, as these flexes correspond to non-euclidean motions in the plane. Below we create a function that finds a basis for the space of these non-trivial motions, if such a space exists.


In [38]:
def infFlex(G, p):
    """
    Calculates a non-trivial infinitesiaml flex inf such a flex exists.

    Parameters
    ----------
    G : list
        Ordered pair consisting of a vertex list and an edge list.
    p : list
        Configuration in d-dimensional Euclidean Space.
    
    Returns
    -------
    flex : list
        A non-trivial infinitesimal flex if one exists, otherwise all zeros.

    """

    verts = G[0]
    edges = G[1]
    n = len(verts)
    E = len(edges)
    d = len(p[verts[0]])

    bars = [p[i]-p[j] for i,j in edges if i<j]

    # Create a list of symbols that will represent our flex
    flex = []
    for i in range(n):
        flex.append(Vector(*[symbols('x' + str(i) + str(k)) for k in range(d)]))

    # Create a list of constraints that fix the first point in the
    # configuration, then constrains the second flex to be in the line between
    # p0 and p1, the third flex to lie in the plane defined by p0, p1 and p2
    # and so on. This is possible if n > dim and the configuration is in
    # general position. We do this by finding the nullspace of the matrix of
    # vectors that define the hyperplane, which gives a normal vector. To
    # ensure the flex remains in the hyperplane, we enforce that the dot
    # product between the normal and the flex is zero.
    nonTrivConstraints = []
    for entry in flex[0]:
        nonTrivConstraints.append(entry)

    p0 = p[0]
    for i in range(1,d):
        vects = [p0 - pi for pi in p[1:i+1]]
        fixedHypPlane = Matrix([v.transpose() for v in vects])
        normal = fixedHypPlane.nullspace()[0]

        constraint = flex[i].dot(normal)
        nonTrivConstraints.append(constraint)

    # Solve the system and apply the constraints to flex
    nonTrivSoln = solve(nonTrivConstraints)
    flex = [delta.subs(nonTrivSoln) for delta in flex]

    # Create the constraints necessary for infinitesimal rigidity
    flexEqns = [flex[i]-flex[j] for i,j in edges if i<j]
    rigidConstraints = [bar.dot(flexEqns[i]) for i, bar in enumerate(bars)]

    # Solve the system
    soln = solve(rigidConstraints, dict=True)
    flex = [delta.subs(soln[0]) for delta in flex]

    return flex

### Examlpes

To see how we can use this function, we look at the following examples:

* Octahedron with a flattened vertex
* Triangle
* Square
* Square with a brace
* Two adjoined squares with braces
* Two adjoined squares with braces but in $\mathbb{R}^3$


In [41]:
# Flattened Octahedron
a1 = Vector(-2,0,0)
a2 = Vector(4,2,0)
a3 = Vector(2,0,0)
a4 = Vector(4,-2,0)
a5 = Vector(1,0,3)
a6 = Vector(1,0,-3)

config = [a1, a2, a3, a4, a5, a6]
nbs = {0: [1, 3, 4, 5], 1: [0, 2, 4, 5], 2: [1, 3, 4, 5],
       3: [0, 2, 4, 5], 4: [0, 1, 2, 3], 5: [0, 1, 2, 3]}
edges = [(i, j) for i in nbs for j in nbs[i] if i < j]
a = infFlex([[0, 1, 2, 3, 4, 5], edges], config)

In [6]:
# Triangle
b1 = Vector(0,0)
b2 = Vector(1,0)
b3 = Vector(0,1)

verts = [b1,b2,b3]
nbs = {0:[1,2], 1:[0,2], 2:[0,1]}
b = infFlex(verts, nbs)

In [7]:
# Square
c1 = Vector(0,0)
c2 = Vector(1,0)
c3 = Vector(0,1)
c4 = Vector(1,1)

verts = [c1,c2,c3,c4]
nbs = {0:[1,2], 1:[0,3], 2:[0,3], 3:[1,2]}
c = infFlex(verts, nbs)

In [8]:
# Square with brace
d1 = Vector(0,0)
d2 = Vector(1,0)
d3 = Vector(0,1)
d4 = Vector(1,1)

verts = [d1,d2,d3,d4]
nbs = {0:[1,2,3], 1:[0,3], 2:[0,3], 3:[0,1,2]}
d = infFlex(verts, nbs)

In [9]:
# Two joined squares with braces
e1 = Vector(0,0)
e2 = Vector(1,0)
e3 = Vector(2,0)
e4 = Vector(0,1)
e5 = Vector(1,1)
e6 = Vector(2,1)

verts = [e1,e2,e3,e4,e5,e6]
nbs = {0:[1,3,4], 1:[0,2,4,5], 2:[1,5], 3:[0,4], 4:[0,1,3,5], 5:[1,2,4]}
e = infFlex(verts, nbs)

In [10]:
# Two joined squares with braces, but in R^3
f1 = Vector(0,0,0)
f2 = Vector(1,0,0)
f3 = Vector(2,0,0)
f4 = Vector(0,1,0)
f5 = Vector(1,1,0)
f6 = Vector(2,1,0)

verts = [f1,f2,f3,f4,f5,f6]
nbs = {0:[1,3,4], 1:[0,2,4,5], 2:[1,5], 3:[0,4], 4:[0,1,3,5], 5:[1,2,4]}
f = infFlex(verts, nbs)

## The Rigidiy Matrix

Recall that we define the matrix of the system of infintesimal flex equations to be the **Rigidity Matrix**. There is a wealth of information we can draw from the properties of this matrix, so we proceed by defining functions to create the Rigidity Matrix, both symbolically and numerically.


In [23]:
def RigidityMatrixNumerical(G, p):
    """
    Create the numerical rigidity matrix of the framework (G,p).

    Parameters
    ----------
    G : list
        Ordered pair consisting of a vertex list and an edge list.
    p : list
        Configuration in d-dimensional Euclidean Space.

    Returns
    -------
    R : ndarray
        The rigidity matrix of (G,p).

    """

    verts = G[0]
    edges = G[1]
    n = len(verts)
    E = len(edges)
    d = len(p[verts[0]])

    R = np.zeros((E, d*n))
    for row, (i, j) in enumerate(edges):
        R[row, i*d:(i+1)*d] = (p[i]-p[j]).transpose()
        R[row, j*d:(j+1)*d] = (p[j]-p[i]).transpose()
    return R

In [21]:
def RigidityMatrixSymbolic(G):
    """
    Create the symbolic rigidity matrix of the framework (G,p).

    Parameters
    ----------
    G : list
        Ordered pair consisting of a vertex list and an edge list.

    Returns
    -------
    R : MutableDenseMatrix
        The rigidity matrix of (G,p).

    """

    verts = G[0]
    edges = G[1]
    n = len(verts)
    E = len(edges)

    R = Matrix.zeros(E, n)
    for row, (i, j) in enumerate(edges):
        R[row, i:i+1] = [symbols('p' + str(i)) - symbols('p' + str(j))]
        R[row, j:j+1] = [symbols('p' + str(j)) - symbols('p' + str(i))]
    return R

In [26]:
a1 = Vector(-2, 0, 0)
a2 = Vector(4, 2, 0)
a3 = Vector(2, 0, 0)
a4 = Vector(4, -2, 0)
a5 = Vector(1, 0, 3)
a6 = Vector(1, 0, -3)

config = [a1, a2, a3, a4, a5, a6]
nbs = {0: [1, 3, 4, 5], 1: [0, 2, 4, 5], 2: [1, 3, 4, 5],
       3: [0, 2, 4, 5], 4: [0, 1, 2, 3], 5: [0, 1, 2, 3]}
edges = [(i, j) for i in nbs for j in nbs[i] if i < j]
a = RigidityMatrixNumerical([[0, 1, 2, 3, 4, 5], edges], config)
a_2 = RigidityMatrixSymbolic([[0, 1, 2, 3, 4, 5], edges])
a_2

Matrix([
[p0 - p1, -p0 + p1,        0,        0,        0,        0],
[p0 - p3,        0,        0, -p0 + p3,        0,        0],
[p0 - p4,        0,        0,        0, -p0 + p4,        0],
[p0 - p5,        0,        0,        0,        0, -p0 + p5],
[      0,  p1 - p2, -p1 + p2,        0,        0,        0],
[      0,  p1 - p4,        0,        0, -p1 + p4,        0],
[      0,  p1 - p5,        0,        0,        0, -p1 + p5],
[      0,        0,  p2 - p3, -p2 + p3,        0,        0],
[      0,        0,  p2 - p4,        0, -p2 + p4,        0],
[      0,        0,  p2 - p5,        0,        0, -p2 + p5],
[      0,        0,        0,  p3 - p4, -p3 + p4,        0],
[      0,        0,        0,  p3 - p5,        0, -p3 + p5]])